In [1]:
import pandas as pd
import numpy as np
import google.generativeai as palm
from utils import VectorEmbeddings, remove_html_tags, remove_req_qual_str, job_categories_to_string
from concurrent.futures import ThreadPoolExecutor, as_completed
import time, json, re, ast
import logging
from datetime import datetime
from pathlib import Path
from typing import Tuple


In [2]:
logging.basicConfig(
    filename="script.log",
    format="%(asctime)s %(message)s",
    filemode="w",
    level=logging.INFO,
)

In [3]:
def job_categories_to_string(value: str) -> str:
    """Transforms the job categories to just the actual occupational series

    Args:
        value (str): The value from the 'job_categories'

    Returns:
        str: string that it returns with the OSCs in one comma separated string

    Examples:
        >>> job_categories_to_string("[{'series': '0808'}, {'series': '0810'}]")
        '0808, 0810'
    """
    series_list = ast.literal_eval(value)
    series_values = [item["series"] for item in series_list]
    try:
        return ", ".join(series_values)
    except:
        return None

In [4]:
def read_raw_data_to_dict(
    path: str = "../../raw_data",
    files: list = None,
    build_teams: bool = False,
    build_terminal_teams: bool = True,
) -> Tuple[dict, list]:
    """Reads in raw data and returns a dict of dataframes and a list of  months.

    Args:
        path (str, optional): Path to the raw data directory. Defaults to "../../raw_data".
        files (list, optional): A list of files you want to read in. Defaults to None.

    Returns:
        Tuple[dict, list]:  dict of data frames, the key is the month, and the value is the dataframe and a list of months sorted
    """

    if not files:
        p = Path(path)
        files = [str(file) for file in p.glob("[0-9]*txt")]
        files.sort()

    df_dict = {}
    team_dicts_by_month = {}
    month_keys = []

    for file in files:
        df = pd.read_table(file, sep="|", encoding="windows-1252", low_memory=False)
        month = str(file).rsplit("/", 1)[1].replace(".txt", "")

        if build_teams:
            if build_terminal_teams:
                team_dicts_by_month[month] = build_team_dict(df)
            else:
                team_dicts_by_month[month] = build_team_dict_with_non_terminal(df)
        df["month"] = month
        month_keys.append(month)

        df_dict[month] = df

    month_keys.sort()

    if build_teams:
        return df_dict, month_keys, team_dicts_by_month
    else:
        return df_dict, month_keys

In [5]:
path = '/Users/bryanadams/github/LaborFlows/usa_jobs/raw_data'
p = Path(path)

In [6]:
files = [str(file) for file in p.glob("[0-9]*txt")]
year_pattern = re.compile(r'(2017|2018|2019|2020)')
matching_files = []
for file in files:
    if re.search(year_pattern, file):
        matching_files.append(file)

matching_files.sort()

In [7]:
df_dict, month_keys = read_raw_data_to_dict(files = matching_files)

In [8]:
osc_count_dict = {}
for people in df_dict.values():
    unique_values, unique_counts = np.unique(people['SERIES'].to_list(), return_counts=True)
    for z in zip(unique_values, unique_counts):
        osc = str(z[0])
        osc = osc.replace('.0','')
        if len(osc) == 2:
            osc = '00'+osc
        if len(osc) == 3:
            osc = '0'+osc
        if osc not in osc_count_dict.keys():
            osc_count_dict[osc] = z[1]
        else:
            osc_count_dict[osc] += z[1]

In [9]:
sorted_dict = dict(sorted(osc_count_dict.items(), key=lambda item: item[1], reverse=True))

In [10]:
sorted_dict.keys()

dict_keys(['1102', '0346', '0801', '0810', '0830', '0301', '0855', '2210', '0340', '0343', '0854', '1670', '1550', '1515', '1910', '0850', '1101', '0861', '2010', '0560', '1301', '1105', '2003', '0401', '0893', '1170', '1712', '1083', '0808', '0501', '2001', '1320', '0802', '0819', '1310', '0896', '1350', '0809', '0803', '0601', '0806', '0391', '0510', '1150', '2130', '0180', '1103', '1520', '1601', '2032', '0403', '0020', '1008', '0413', '1382', '0028', '1640', '1384', '0858', '1152', '0193', '2181', '1529', '0505', '1373', '1171', '0804', '0807', '0062', '1306', '0018', '1176', '1313', '1340', '1370', '0905', '0856', '0405', '1315', '0486', '0602', '1321', '2101', '0408', '1371', '1316', '0630', '0871', '0415', '0511', '0690', '0899', '0610', '0414', '0404', '0150', '1374', '0132', '1360', '0080', '0460', '0817', '1311', '1530', '0482', '0840', '1399', '0089', '1130', '0nan', '0110', '0101', '2150', '1501', '2030', '2005', '0170', '0561', '0308', '0181', '0201', '0341', '1801', '51C0

In [6]:
top_15_osc = ['1102', '0346', '0801', '0810', '0830', '0301', '0855', '2210', '0340', '0343', '0854', '1670', '1550', '1515', '1910']

In [7]:
knowledge_df = pd.read_excel('../onet_competencies/knowledge_competencies.xlsx', skiprows=3)

In [8]:
with open('../.api_key', 'r') as file:
    api_key = file.readline()
vector_embeddings = VectorEmbeddings(api_key=api_key)

In [9]:
job_df = pd.read_json('job_descriptions.json', lines=True)

In [10]:
job_df['osc'] = job_df['job_categories'].apply(job_categories_to_string)

In [11]:
job_ids = []
for i, row in job_df.iterrows():
    if row['osc']:
        if ',' not in row['osc']:
            if row['osc'] in top_15_osc:
                job_ids.append(row['usajobs_control_number'])

In [12]:
len(job_ids)

46804

In [13]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
text_model = text_models[0]

In [14]:
# An example high level requirement for this knowledge is {} while an example low level requirement for this knowledge is {}.

def make_ksa_prompt(k_skill, k_description, job_ad):
    query = 'Does this job ad require {} knowledge to perform the job?'.format(k_skill)
    leading_text = '{} is the {}. Look at the job ad and decide if {} is required to perform this job. If {} is required please provide the specific tasks in a bulletized format from the job ad that relate to {}, if not simply say no'.format(k_skill, k_description,k_skill, k_skill, k_skill)
    prompt = vector_embeddings.make_prompt(leading_text, query, job_ad)
    
    return prompt

In [15]:
def generate_answers(prompt, text_model, candidate_count: int = 3, temperature:float = 0.5):
    count = 0
    while count<4:
        try:
            answers = palm.generate_text(prompt=prompt,
                                    model=text_model,
                                    candidate_count=candidate_count,
                                    temperature=temperature,
                                    max_output_tokens=1000)
            count = 10
        except:
            answers = 'FAILED'
            logging.info("FAILED")
            count += 1
            time.sleep((count+1)**2*6)
    return answers

In [16]:
def ksa_answers(ksa,prompt,**kwargs):
    return_dict = {ksa:generate_answers(prompt,**kwargs)}
    return return_dict

In [17]:
def determine_if_ksa_is_present(answers):
    for answer in answers.candidates:
        if answer['output'].lower().startswith('no'):
            return False
    return True

## Make a dict of the prompts

In [18]:
run_df = job_df[job_df['usajobs_control_number'].isin(job_ids[51:1000])]

In [19]:
job_prompts = {}
for job_index, job_row in run_df.iterrows():
    job_prompts[job_row['usajobs_control_number']] = {}
    job = remove_req_qual_str(remove_html_tags(job_row['requirements_qualifications']))
    for ksa_index, ksa_row in knowledge_df.iterrows():
        job_prompts[job_row['usajobs_control_number']][ksa_row['Element Name']] = make_ksa_prompt(ksa_row['Element Name'].lower(), ksa_row['Element Description'].lower(), job)

In [21]:
job_id_ksa_dict = {}
for job_id , ksa_prompts in job_prompts.items():
    job_responses = []
    response_list = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        for ksa, prompt in ksa_prompts.items():
            job_responses.append(executor.submit(ksa_answers, ksa=ksa, 
                                                prompt = prompt, 
                                                text_model = text_model,
                                                temperature = 0))
    for body in as_completed(job_responses):
        response_list.append(body.result())

    ksa_list = []
    for response in response_list:
        for ksa, answer in response.items():
            if answer == 'FAILED':
                ksa_list.append(ksa+'FAILED')
            elif determine_if_ksa_is_present(answer):
                ksa_list.append(ksa)

    job_id_ksa_dict[job_id] = ksa_list
    logging.info("Finished: {}".format(job_id))
    time.sleep(20)

In [22]:
with open('ksa_run_three.json', 'w') as json_file:
    json.dump(job_id_ksa_dict, json_file)

In [36]:
ksa_list = []
for response in response_list:
    for ksa, answer in response.items():
        if determine_if_ksa_is_present(answer):
            ksa_list.append(ksa)

In [37]:
ksa_list

['English Language',
 'Law and Government',
 'Economics and Accounting',
 'Education and Training',
 'Business and Management',
 'Administration and Management']

In [ ]:
with ThreadPoolExecutor(max_workers=5) as executor:
    for i, row in job_df.head(n=2).iterrows():
        print(row['usajobs_control_number'])       
        job_responses.append(executor.submit(determine_ksa_vector, job_id = row['usajobs_control_number'], job = row['requirements_qualifications'], ksa_df = knowledge_df, text_model=text_model))
    for body in as_completed(job_responses):
        response_list.append(body.result())

In [14]:
job_prompts.keys()

dict_keys([514962300, 514177900, 540927700, 534312000, 546156800])

In [56]:
def determine_ksa_vector(job_id, job, ksa_df, text_model, **kwargs):
    ksa = []
    non_ksa = []
    return_dict = {}
    # for i, row in knowledge_df.iterrows():
    for i, row in ksa_df.iterrows():
        prompt = make_ksa_prompt(row['Element Name'].lower(), row['Element Description'].lower(), job)

        answers = generate_answers(prompt, text_model, **kwargs)

        add = determine_if_ksa_is_present(answers)

        if add:
            ksa.append(row['Element Name'])
        else:
            non_ksa.append(row['Element Name'])

    return_dict[job_id] = {'ksa': ksa, 'non_ksa': non_ksa}
    
    return return_dict

In [ ]:
# job_responses = [] 
# response_list = []
# with ThreadPoolExecutor(max_workers=10) as executor:
#     for file in test_list:
#         print('yes')       
#         job_responses.append(executor.submit(determine_ksa_vector, file, ksa_df = knowledge_df, text_model=text_model))
#     for body in as_completed(job_responses):
#         response_list.append(body.result())

In [57]:
job_responses = [] 
response_list = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for i, row in job_df.head(n=2).iterrows():
        print(row['usajobs_control_number'])       
        job_responses.append(executor.submit(determine_ksa_vector, job_id = row['usajobs_control_number'], job = row['requirements_qualifications'], ksa_df = knowledge_df, text_model=text_model))
    for body in as_completed(job_responses):
        response_list.append(body.result())

514962300
514177900


In [12]:
determine_ksa_vector(job, knowledge_df, text_model)

(['Administration and Management',
  'Engineering and Technology',
  'Engineering and Technology',
  'Mathematics and Science',
  'Mathematics',
  'Education and Training'],
 ['Business and Management',
  'Administrative',
  'Economics and Accounting',
  'Sales and Marketing',
  'Customer and Personal Service',
  'Personnel and Human Resources',
  'Manufacturing and Production',
  'Production and Processing',
  'Food Production',
  'Computers and Electronics',
  'Design',
  'Building and Construction',
  'Mechanical',
  'Physics',
  'Chemistry',
  'Biology',
  'Psychology',
  'Sociology and Anthropology',
  'Geography',
  'Health Services',
  'Medicine and Dentistry',
  'Therapy and Counseling',
  'Arts and Humanities',
  'English Language',
  'Foreign Language',
  'Fine Arts',
  'History and Archeology',
  'Philosophy and Theology',
  'Law and Public Safety',
  'Public Safety and Security',
  'Law and Government',
  'Communications',
  'Telecommunications',
  'Communications and Media

In [15]:
determine_ksa_vector(job, knowledge_df, text_model)

(['Engineering and Technology',
  'Engineering and Technology',
  'Mathematics and Science',
  'Mathematics',
  'Education and Training',
  'Public Safety and Security'],
 ['Business and Management',
  'Administration and Management',
  'Administrative',
  'Economics and Accounting',
  'Sales and Marketing',
  'Customer and Personal Service',
  'Personnel and Human Resources',
  'Manufacturing and Production',
  'Production and Processing',
  'Food Production',
  'Computers and Electronics',
  'Design',
  'Building and Construction',
  'Mechanical',
  'Physics',
  'Chemistry',
  'Biology',
  'Psychology',
  'Sociology and Anthropology',
  'Geography',
  'Health Services',
  'Medicine and Dentistry',
  'Therapy and Counseling',
  'Arts and Humanities',
  'English Language',
  'Foreign Language',
  'Fine Arts',
  'History and Archeology',
  'Philosophy and Theology',
  'Law and Public Safety',
  'Law and Government',
  'Communications',
  'Telecommunications',
  'Communications and Media

In [144]:
ksa, non_ksa

(['Administration and Management',
  'Engineering and Technology',
  'Engineering and Technology',
  'Design',
  'Building and Construction',
  'Mathematics and Science',
  'Mathematics',
  'Physics',
  'Geography',
  'Medicine and Dentistry',
  'Therapy and Counseling',
  'Education and Training',
  'English Language'],
 ['Business and Management',
  'Administrative',
  'Economics and Accounting',
  'Sales and Marketing',
  'Customer and Personal Service',
  'Personnel and Human Resources',
  'Manufacturing and Production',
  'Production and Processing',
  'Food Production',
  'Computers and Electronics',
  'Mechanical',
  'Chemistry',
  'Biology',
  'Psychology',
  'Sociology and Anthropology',
  'Health Services',
  'Arts and Humanities',
  'Foreign Language',
  'Fine Arts',
  'History and Archeology',
  'Philosophy and Theology',
  'Law and Public Safety',
  'Public Safety and Security',
  'Law and Government',
  'Communications',
  'Telecommunications',
  'Communications and Media

In [142]:
ksa, non_ksa

(['Administration and Management',
  'Engineering and Technology',
  'Engineering and Technology',
  'Design',
  'Building and Construction',
  'Mathematics and Science',
  'Mathematics',
  'Physics',
  'Geography',
  'Medicine and Dentistry',
  'Therapy and Counseling',
  'Education and Training',
  'English Language'],
 ['Business and Management',
  'Administrative',
  'Economics and Accounting',
  'Sales and Marketing',
  'Customer and Personal Service',
  'Personnel and Human Resources',
  'Manufacturing and Production',
  'Production and Processing',
  'Food Production',
  'Computers and Electronics',
  'Mechanical',
  'Chemistry',
  'Biology',
  'Psychology',
  'Sociology and Anthropology',
  'Health Services',
  'Arts and Humanities',
  'Foreign Language',
  'Fine Arts',
  'History and Archeology',
  'Philosophy and Theology',
  'Law and Public Safety',
  'Public Safety and Security',
  'Law and Government',
  'Communications',
  'Telecommunications',
  'Communications and Media

In [140]:
ksa, non_ksa

(['Administration and Management',
  'Engineering and Technology',
  'Engineering and Technology',
  'Design',
  'Building and Construction',
  'Mathematics and Science',
  'Mathematics',
  'Physics',
  'Geography',
  'Medicine and Dentistry',
  'Therapy and Counseling',
  'Education and Training'],
 ['Business and Management',
  'Administrative',
  'Economics and Accounting',
  'Sales and Marketing',
  'Customer and Personal Service',
  'Personnel and Human Resources',
  'Manufacturing and Production',
  'Production and Processing',
  'Food Production',
  'Computers and Electronics',
  'Mechanical',
  'Chemistry',
  'Biology',
  'Psychology',
  'Sociology and Anthropology',
  'Health Services',
  'Arts and Humanities',
  'English Language',
  'Foreign Language',
  'Fine Arts',
  'History and Archeology',
  'Philosophy and Theology',
  'Law and Public Safety',
  'Public Safety and Security',
  'Law and Government',
  'Communications',
  'Telecommunications',
  'Communications and Media

In [112]:
prompt = make_ksa_prompt(knowledge_df.iloc[3]['Element Name'].lower(), knowledge_df.iloc[3]['Element Description'].lower())

In [119]:
temperature = 0.5
answers = palm.generate_text(prompt=prompt,
                            model=text_model,
                            candidate_count=5,
                            temperature=temperature,
                            max_output_tokens=1000)
for i, candidate in enumerate(answer.candidates):
  print(f"Candidate {i}: {candidate['output']}\n")

Candidate 0: No

Candidate 1: No

Candidate 2: No, transportation is not required to perform the job.

Candidate 3: No

Candidate 4: No



In [126]:
for answer in answers.candidates:
    if answer['output'].lower().startswith('no'):
        print('yes')

yes
yes
yes
yes
yes


In [36]:
for i, candidate in enumerate(answer.candidates):
  print(f"Candidate {i}: {candidate['output']}\n")

In [110]:
knowledge_df.iloc[2]['Element Description']

'Knowledge of administrative and office procedures and systems such as word processing, managing files and records, stenography and transcription, designing forms, and workplace terminology.'